## Annotate probe IDs with gene symbols 

In order to identify the genes that compounds with high MAS and low TAS influence

In [1]:
import pathlib
import pandas as pd
import numpy as np

In [2]:
top_n_cpds = 6
gene_cut = 2

In [3]:
# Load mapping resource
# Downloaded from: http://amp.pharm.mssm.edu/public/L1000CDS_download/
url = "http://amp.pharm.mssm.edu/public/L1000CDS_download/apiRowMeta.json"
map_df = pd.read_json(url)

# Setup a dictionary to rename the map
updater = dict(zip(map_df.pr_id, map_df.pr_gene_symbol))

print(map_df.shape)
map_df.head()

(36590, 10)


,is_lm,is_l1000,pr_id,pr_gene_id,pr_gene_title,ds_index,pr_gene_symbol,_id,is_bing,pr_pool_id
0,0.0,True,202938_x_at,100510314///100510451///27341///91695,-666,10985.0,-666,51438660dfe7719b8846fd9c,0.0,[inferred]
1,0.0,True,204006_s_at,2214///2215,-666,11270.0,-666,51438660dfe7719b8846feb9,0.0,[inferred]
2,0.0,True,204060_s_at,5613///5616,-666,11287.0,-666,51438660dfe7719b8846feca,0.0,[inferred]
3,0.0,True,204419_x_at,3047///3048,-666,11399.0,-666,51438660dfe7719b8846ff3a,0.0,[inferred]
4,0.0,True,204438_at,414308///4360,-666,11405.0,-666,51438660dfe7719b8846ff40,0.0,[inferred]


In [4]:
# Load activity scores
file = pathlib.Path("../5.paper_figures/data/highmas_lowtas_compounds.tsv")
activity_df = pd.read_csv(file, sep="\t")

print(activity_df.shape)
activity_df.head(3)

(126, 5)


,cpd,mas_mean,tas_mean,cpd_count,mas_tas_dff
0,alisertib,0.709348,0.200119,60,0.509229
1,dasatinib,0.725527,0.247690,60,0.477837
2,aphidicolin,0.706906,0.267356,60,0.439550


In [5]:
# What are the top compounds that change lots of MAS but not TAS
top_cpds = activity_df.head(top_n_cpds).cpd.tolist()
top_cpds

['alisertib',
 'dasatinib',
 'aphidicolin',
 'at13387',
 'sta-5326',
 'combretastatin-a-4']

In [6]:
# Load L1000 data to obtain high differential genes
file = pathlib.Path("Consensus/L1000/moa_sizes_consensus_datasets/modz_level5_data.csv")
df = pd.read_csv(file)

df = df.query("pert_iname in @top_cpds").reset_index(drop=True)

print(df.pert_iname.value_counts())
print(df.shape)
df.head(2)

aphidicolin           6
combretastatin-a-4    6
dasatinib             6
sta-5326              6
at13387               6
alisertib             6
Name: pert_iname, dtype: int64
(36, 980)


,sig_id,200814_at,222103_at,201453_x_at,204131_s_at,200059_s_at,205067_at,213702_x_at,214435_x_at,201334_s_at,...,212536_at,218529_at,211071_s_at,203341_at,205379_at,pert_id,pert_idose,dose,pert_iname,moa
0,REP.A013_A549_24H:K01,-0.556544,-0.397997,0.524216,0.198366,-0.178928,-0.415482,7.722396,-0.218728,1.083014,...,0.321159,-1.686505,1.521019,-0.874401,-1.242169,BRD-K41783120,10 uM,6,sta-5326,interleukin synthesis inhibitor
1,REP.A013_A549_24H:K02,-0.183263,-0.448801,-0.326889,2.079945,-0.164152,0.159238,4.571975,-0.449638,1.801585,...,-1.247952,-2.503231,1.810855,-1.152574,-0.810116,BRD-K41783120,3.33 uM,5,sta-5326,interleukin synthesis inhibitor


In [15]:
# Obtain background gene lists
background_df = pd.DataFrame(
    df.columns[df.columns.str.endswith("_at")],
    columns=["probe"]
)

background_df = background_df.assign(gene_symbol=background_df.probe.replace(updater))

output_file = pathlib.Path("results", "background_gene_list.tsv")
background_df.to_csv(output_file, sep="\t", index=False)

background_df.head()

,probe,gene_symbol
0,200814_at,PSME1
1,222103_at,ATF1
2,201453_x_at,RHEB
3,204131_s_at,FOXO3
4,200059_s_at,RHOA


In [7]:
expression_df = (
    df
    .groupby(["pert_iname", "moa"])
    .median()
    .reset_index()
    .melt(
        id_vars=["pert_iname", "moa"],
        value_vars=df.columns[df.columns.str.endswith("_at")],
        value_name="L1000_readout",
        var_name="L1000_probe"
    )
)

expression_df = (
    expression_df
    .assign(L1000_abs_readout = expression_df.L1000_readout.abs())
    .query("L1000_abs_readout > @gene_cut")
    .sort_values(by="pert_iname")
    .reset_index(drop=True)
)

expression_df = expression_df.assign(gene_symbol=expression_df.L1000_probe.replace(updater))

output_file = pathlib.Path("results", "differential_mas_vs_tas_genes.tsv")
expression_df.to_csv(output_file, sep="\t", index=False)

print(expression_df.shape)
expression_df.head()

(330, 6)


,pert_iname,moa,L1000_probe,L1000_readout,L1000_abs_readout,gene_symbol
0,alisertib,aurora kinase inhibitor,208708_x_at,-3.419625,3.419625,EIF5
1,alisertib,aurora kinase inhibitor,208728_s_at,-2.298975,2.298975,CDC42
2,alisertib,aurora kinase inhibitor,215903_s_at,2.360950,2.360950,MAST2
3,alisertib,aurora kinase inhibitor,205039_s_at,4.639550,4.639550,IKZF1
4,alisertib,aurora kinase inhibitor,202705_at,-2.406725,2.406725,CCNB2


In [8]:
# Which genes are consistently implicated?
gene_count_df = (
    expression_df
    .gene_symbol
    .value_counts()
    .reset_index()
    .rename({"index": "gene", "gene_symbol": "cpd_count"}, axis="columns")
)

gene_count_df.head(10)

,gene,cpd_count
0,NRIP1,5
1,DLD,4
2,IARS2,3
3,CDC45,3
4,CCNB2,3
5,PKIG,3
6,PPP2R3C,3
7,HMOX1,3
8,GNAS,3
9,EIF5,3
